# 🧠 Qwen2.5 Brain MRI Abnormality Detection

Locate abnormal regions in MRI slices using Qwen2.5 and evaluate with mAP.

## 1. Setup & Imports

In [ ]:
import os, json, base64
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

def encode_image_to_data_uri(path: str) -> str:
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode('utf-8')
    return f"data:image/png;base64,{b64}"

DATASET_DIR = "VLM-Seminar25-Dataset/nova_brain"
IMAGES_DIR = os.path.join(DATASET_DIR, "images")
ANNOT_PATH = os.path.join(DATASET_DIR, "annotations.json")
RESULTS_DIR = "../results/nova_brain"
os.makedirs(RESULTS_DIR, exist_ok=True)

with open(ANNOT_PATH, "r") as f:
    annotations = json.load(f)
case_ids = list(annotations.keys())

load_dotenv(dotenv_path="config/user.env")
api_key = os.environ.get("NEBIUS_API_KEY")
client = OpenAI(base_url="https://api.studio.nebius.com/v1/", api_key=api_key)

In [ ]:
do_new_inference = False

## 2. Model Inference

In [ ]:
detection_results = []
if do_new_inference:
    for case_id in tqdm(case_ids):
        case = annotations[case_id]
        for img_name, img_info in case.get("image_findings", {}).items():
            img_path = os.path.join(IMAGES_DIR, img_name)
            data_uri = encode_image_to_data_uri(img_path)
            prompt = "Please locate any abnormal areas in the MRI image and output the bounding boxes as [x1, y1, x2, y2]."
            completion = client.chat.completions.create(
                model="Qwen/Qwen2.5-VL-72B-Instruct",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": data_uri}},
                        ],
                    }
                ],
            )
            pred = completion.choices[0].message.content.strip()
            detection_results.append({
                "case_id": case_id,
                "image": img_name,
                "prediction": pred
            })

## 3. Save Model Predictions

In [ ]:
with open(os.path.join(RESULTS_DIR, "qwen2.5_detection_results.json"), "w") as f:
    json.dump(detection_results, f, indent=2)
print("Saved detection results.")

Load results

In [ ]:
with open(os.path.join(RESULTS_DIR, "qwen2.5_detection_results.json"), "r") as f:
    detection_results = json.load(f)
print(f"Number of detections: {len(detection_results)}")

## 4. Evaluation & Metrics

In [ ]:
import subprocess
import re

result = subprocess.run(
    ["python", "VLM-Seminar25-Dataset/scripts/calculate_map.py"],
    capture_output=True, text=True
)
print(result.stdout)

m = re.search(r"mAP\s*[:=]\s*([0-9.]+)", result.stdout)
if m:
    mAP = float(m.group(1))
    eval_metrics = {"mAP": mAP}
    with open(os.path.join(RESULTS_DIR, "detection_eval_metrics.json"), "w") as f:
        json.dump(eval_metrics, f, indent=2)
    print("Saved mAP metric.")
    plt.figure(figsize=(4,4))
    plt.bar(["mAP"], [mAP], color="mediumseagreen")
    plt.ylim(0, 1)
    plt.title("Qwen2.5 MRI Detection mAP")
    plt.text(0, mAP + 0.02, f"{mAP:.2f}", ha='center', fontsize=12)
    plt.savefig(os.path.join(RESULTS_DIR, "detection_metrics.png"))
    plt.show()
    plt.close()
    print("Saved mAP plot.")
else:
    print("Could not parse mAP from evaluation output.")